# 使用SFTTrainer进行监督微调

本笔记本展示了如何使用`trl`库中的`SFTTrainer`对`HuggingFaceTB/SmolLM2-135M`模型进行微调。运行笔记本中的单元格将对模型进行微调。您可以通过尝试不同的数据集来选择适合自己的难度。
<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>练习：使用SFTTrainer对SmolLM2进行微调</h2>
    <p>从Hugging Face hub中获取一个数据集，并在该数据集上对模型进行微调。</p> 
    <p><b>难度等级</b></p>
    <p>🐢 使用HuggingFaceTB/smoltalk数据集。</p>
    <p>🐕 尝试使用bigcode/the-stack-smol数据集，并在其特定子集data/python上对代码生成模型进行微调。</p>
    <p>🦁 选择一个与您感兴趣的的数据集进行微调。</p>
</div>

In [ ]:
# Install the requirements in Google Colab
# !pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face

from huggingface_hub import login
login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

In [1]:
# Import necessary libraries
import os
import torch
import warnings
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
warnings.filterwarnings('ignore')

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

/home/wjh/anaconda3/envs/slm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


加载基础模型**Smo1LM2-135M**

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
# 从本地路径加载模型
model_path = "/home/wjh/HFace/models/SmolLM2-135M/"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_path
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_path)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# 使用基础模型生成
在这里，我们将尝试使用没有经过聊天模板微调的基础模型。

In [3]:
# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
user
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a


## 数据集准备
我们将加载一个样本数据集，并将其格式化为训练所需的形式。数据集应以输入-输出对的形式结构化，其中每个输入是一个提示，输出是模型预期给出的回应。

TRL将根据模型的聊天模板格式化输入消息。它们需要以字典列表的形式表示，字典的键包括：role（角色）和content（内容）。

如果您的数据集格式不是TRL可以转换为聊天模板的格式，您需要对其进行处理，请参考`chat_templates_example.ipynb`。

In [4]:
# Load a sample dataset
from datasets import load_dataset

# TODO: define your dataset and config using the path and name parameters
#ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")
# 加载提前离线的smoltalk数据
dataset = load_dataset("parquet", data_files={'train': '/home/wjh/HFace/dataset/smoltalk/everyday-conversations/train-00000-of-00001.parquet',
                                              'test': '/home/wjh/HFace/dataset/smoltalk/everyday-conversations/test-00000-of-00001.parquet'})

In [5]:
dataset['train']['messages'][0]

[{'content': 'Hi there', 'role': 'user'},
 {'content': 'Hello! How can I help you today?', 'role': 'assistant'},
 {'content': "I'm looking for a beach resort for my next vacation. Can you recommend some popular ones?",
  'role': 'user'},
 {'content': "Some popular beach resorts include Maui in Hawaii, the Maldives, and the Bahamas. They're known for their beautiful beaches and crystal-clear waters.",
  'role': 'assistant'},
 {'content': 'That sounds great. Are there any resorts in the Caribbean that are good for families?',
  'role': 'user'},
 {'content': 'Yes, the Turks and Caicos Islands and Barbados are excellent choices for family-friendly resorts in the Caribbean. They offer a range of activities and amenities suitable for all ages.',
  'role': 'assistant'},
 {'content': "Okay, I'll look into those. Thanks for the recommendations!",
  'role': 'user'},
 {'content': "You're welcome. I hope you find the perfect resort for your vacation.",
  'role': 'assistant'}]

## 配置SFTTrainer
SFTTrainer通过各种参数进行配置，以控制训练过程。这些参数包括训练步数、批量大小、学习率和评估策略。请根据您的具体需求和计算资源调整这些参数。(!如果显存不足可以适当减小batch_size)

In [6]:
# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-Instruct"
finetune_tags = ["smol-course", "module_1"]

In [8]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,                 # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    max_seq_length=1024,            # If `None`, it uses the smaller value between `tokenizer.model_max_length` and `1024`.
    learning_rate=5e-5,             # Common starting point for fine-tuning
    logging_steps=10,               # Frequency of logging training metrics
    save_steps=100,                 # Frequency of saving model checkpoints
    eval_strategy="steps",          # Evaluate the model at regular intervals
    eval_steps=100,                 # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id=finetune_name,     # Set a unique name for your model
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=dataset["train"],
    processing_class=tokenizer,
    eval_dataset=dataset["test"],
)
# TODO: 🦁 🐕 将SFTTrainer的参数与所选数据集对齐。例如，如果您使用的是bigcode/the-stack-smol数据集，您需要选择content列。

Map: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2260/2260 [00:00<00:00, 3070.61 examples/s]


## 训练模型
现在训练器已经配置好，我们可以开始训练模型了。训练过程将包括遍历数据集、计算损失，并更新模型的参数以最小化这个损失。

In [9]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")
# Push to hub
#trainer.push_to_hub(tags=finetune_tags)

Step,Training Loss,Validation Loss
100,0.533000,1.089426
200,0.502300,1.060622
300,0.412300,1.041032
400,0.433500,1.031524
500,0.425000,1.025641
600,0.363600,1.032825
700,0.348900,1.032752
800,0.373000,1.031070
900,0.339600,1.038888
1000,0.359100,1.040334


<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>使用微调后的模型进行生成</h2>
    <p>🐕 使用微调后的模型生成一个回应，就像基础示例中那样。</p>
</div>

In [14]:
# Test the fine-tuned model on the same prompt
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# use the fine-tuned to model generate a response, just like with the base example.
outputs = model.generate(**inputs, max_new_tokens=100)
print("After training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

After training:
user
Write a haiku about programming
assistant
I'm a language model, and I'm looking for some programming haikus. Do you have any suggestions?

Yes, I can suggest some. One popular one is "Hello World" by John Gruber. It's a classic and easy to learn.
user
That sounds great. What's the main idea of the poem?
assistant
The main idea of "Hello World" is to introduce the concept of a program and its main


## 💐 你完成了！
这个笔记本提供了一个使用SFTTrainer对HuggingFaceTB/SmolLM2-135M模型进行微调的逐步指南。按照这些步骤，你可以使模型更有效地执行特定任务。